In [ ]:
#| default_exp format
%load_ext autoreload
%autoreload 2

In [ ]:
# | hide
import sys,os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path.cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')


In [ ]:
# | export
import re
from typing import Iterable, Tuple, Union, List

import pandas as pd
import numpy as np
from fastcore.utils import listify
from fastcore.xtras import Path
from geopy.distance import geodesic
from rich.progress import Progress
from pyarrow import ArrowInvalid
from dotenv import load_dotenv, find_dotenv


from extracao.constants import BW, BW_pattern, APP_ANALISE_PT, APP_ANALISE_EN

RE_BW = re.compile(BW_pattern)
MAX_DIST = 10  # Km
LIMIT_FREQ = 84812.50
load_dotenv(find_dotenv(), override=True)

True

In [ ]:
# | export
def _read_df(folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Lê o dataframe formado por folder / stem.[parquet.gzip | fth | xslx]"""
    file = Path(f"{folder}/{stem}.parquet.gzip")
    try:
        df = pd.read_parquet(file)
    except (ArrowInvalid, FileNotFoundError) as e:
        raise e(f"Error when reading {file}") from e
    return df


# Formatação

> Este módulo possui funções auxiliares de formatação dos dados das várias fontes.


In [ ]:
# | export
def parse_bw(
    bw: str,  # Designação de Emissão (Largura + Classe) codificada como string
) -> Tuple[str, str]:    # Largura e Classe de Emissão
    """Parse the bandwidth string"""
    if match := re.match(RE_BW, bw):
        multiplier = BW[match[2]]
        if mantissa := match[3]:
            number = float(f"{match[1]}.{mantissa}")
        else:
            number = float(match[1])
        classe = match[4]
        return str(multiplier * number), str(classe)
    return "-1", "-1"


In [ ]:
# | export
def _filter_matlab(
    df: pd.DataFrame,  # Arquivo de Dados Base de Entrada
) -> pd.DataFrame:  # Arquivo de Dados formatado para leitura no Matlab
    """Recebe a base de dados da Anatel e formata as colunas para leitura de acordo com os requisitos do Matlab"""
    df["#Estação"] = df["Número_Estação"]
    df.loc[df.Multiplicidade != "1", "#Estação"] = (
        df.loc[df.Multiplicidade != "1", "Número_Estação"]
        + "+"
        + df.loc[df.Multiplicidade != "1", "Multiplicidade"]
    )
    cols_desc = [
        "Fonte",
        "Status",
        "Classe",
        "Entidade",
        "Fistel",
        "#Estação",
        "Município_IBGE",
        "UF",
    ]
    df.loc[:, cols_desc].fillna("NI", inplace=True)

    df["Descrição"] = (
        "["
        + df.Fonte
        + "] "
        + df.Status
        + ", "
        + df.Classe
        + ", "
        + df.Entidade.str.title()
        + " ("
        + df.Fistel
        + ", "
        + df["#Estação"]
        + "), "
        + df.Município_IBGE
        + "/"
        + df.UF
    )

    bad_coords = df.Coords_Valida_IBGE == "0"

    df.loc[bad_coords, "Descrição"] = df.loc[bad_coords, "Descrição"] + "*"

    df.loc[bad_coords, ["Latitude", "Longitude"]] = df.loc[
        bad_coords, ["Latitude_IBGE", "Longitude_IBGE"]
    ].values

    df = df.loc[:, APP_ANALISE_PT]
    df.columns = APP_ANALISE_EN
    return df

In [ ]:
# | export
def _format_matlab(
    df: pd.DataFrame,  # Arquivo de Dados Base de Entrada
) -> pd.DataFrame:  # Arquivo de Dados formatado para leitura no Matlab
    """Formata o arquivo final de dados para o formato esperado pela aplicação em Matlab"""
    df = df.astype("string")
    df.loc[len(df), :] = [
        "-1",
        "-15.7801",
        "-47.9292",
        "[TEMP] L, FX, Estação do SMP licenciada (cadastro temporário)",
        "10",
        "999999999",
        "NI",
        "-1",
    ]  # Paliativo...
    for c in ["Latitude", "Longitude"]:
        df[c] = df[c].fillna(-1).astype("float32")
    df["Frequency"] = df["Frequency"].astype("float64")
    df.loc[df.Service.isin(["", "-1"]), "Service"] = pd.NA
    df["Service"] = df.Service.fillna("-1").astype("int16")
    df.loc[df.Station.isin(["", "-1"]), "Station"] = pd.NA
    df["Station"] = df.Station.fillna("-1").astype("int32")
    df.loc[df.BW.isin(["", "-1"]), "BW"] = pd.NA
    df["BW"] = df["BW"].astype("float32").fillna(-1)
    df.loc[df["Class"].isin(["", "-1"]), "Class"] = pd.NA
    df["Class"] = df.Class.fillna("NI").astype("category")
    df = df[df.Frequency <= LIMIT_FREQ]
    df.sort_values(
        by=["Frequency", "Latitude", "Longitude", "Description"], inplace=True
    )
    # unique_columns = ["Frequency", "Latitude", "Longitude"]
    # df = df.drop_duplicates(subset=unique_columns, keep="last").reset_index(drop=True)
    df["Id"] = [f"#{i+1}" for i in df.index]
    df["Id"] = df.Id.astype("string")
    df.loc[df.Description == "", "Description"] = pd.NA
    df["Description"] = df["Description"].astype("string").fillna("NI")
    return df[["Id"] + list(APP_ANALISE_EN)]

### Mesclagem
Função auxiliar para mesclar registros que são iguais das diversas bases, i.e. estão a uma distância menor que `MAX_DIST` e verificar a validade da mesclagem

In [ ]:
# | export
def get_km_distance(row):
    return geodesic((row[0], row[1]), (row[2], row[3])).km


def merge_on_frequency(
    df_left: pd.DataFrame,  # DataFrame da esquerda a ser mesclado
    df_right: pd.DataFrame,  # DataFrame da direira a ser mesclado
    on: str = "Frequency",  # Coluna usada como chave de mesclagem
    coords: Tuple[str] = ("Latitude", "Longitude"),
    description: str = "Description",
    suffixes: Tuple[str] = ("_x", "_y"),  # Sufixo para as colunas que foram criadas
) -> pd.DataFrame:  # DataFrame resultante da mesclagem
    """Mescla os dataframes baseados na frequência
    É assumido que as colunas de ambos são idênticas, caso contrário os filtros não irão funcionar como esperado
    """
    df: pd.DataFrame = pd.merge(
        df_left.astype("string"),
        df_right.astype("string"),
        on=on,
        how="outer",
        suffixes=suffixes,
        indicator=True,
        copy=False,
    )

    x, y = suffixes
    lat, long = coords


    left = df._merge == "left_only"
    right = df._merge == "right_only"
    both = df._merge == "both"
    df = df.drop(columns=["_merge"])
    
    left_cols: List[str] = [c for c in df.columns if y not in c]
    right_cols: List[str] = [c for c in df.columns if x not in c]


    only_left = df.loc[left, left_cols].drop_duplicates().reset_index(drop=True)
    only_left.columns = [c.removesuffix(x) for c in left_cols]

    only_right = df.loc[right, right_cols].drop_duplicates().reset_index(drop=True)
    only_right.columns = [c.removesuffix(y) for c in right_cols]

    both_columns = [f"{lat}{x}", f"{long}{x}", f"{lat}{y}", f"{long}{y}"]

    df.loc[both, "Distance"] = df.loc[both, both_columns].apply(get_km_distance, axis=1)

    close = df.loc[both, "Distance"] <= MAX_DIST
    df_close = df.loc[(both & close)].drop_duplicates().reset_index(drop=True)
    df_close[f"{description}{x}"] = (
        df_close[f"{description}{x}"] + " | " + df_close[f"{description}{y}"]
    )
    df_close = df_close[left_cols]
    df_close.columns = only_left.columns

    df_far_left = (
        df.loc[(both & ~close), left_cols].drop_duplicates().reset_index(drop=True)
    )
    df_far_left.columns = only_left.columns

    df_far_right = (
        df.loc[(both & ~close), right_cols].drop_duplicates().reset_index(drop=True)
    )
    df_far_right.columns = only_right.columns

    merged_df = pd.concat(
        [df_left, df_right, df_close, df_far_right, df_far_left], ignore_index=True
    )    
    return merged_df.astype('string')

In [ ]:
folder = Path.cwd().parent / "dados"

In [ ]:
df_left = _filter_matlab(pd.read_parquet(folder / 'base.parquet.gzip'))

In [ ]:
df_left.tail()

,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
821186,89.1,-20.320200000000000,-40.322200000000000,"[MOS] FM-C4, A3, Radio E Televisao Espirito Sa...",230,1015028567,<NA>,256.0
821187,91.9,-29.7950194444443333,-50.0380000000000000,"[MOS] FM-C4, B1, Plus Radiodifusao Ltda (50409...",230,699673097,<NA>,256.0
821188,92.5,-21.457500000000000,-43.553800000000000,"[MOS] FM-C7, A1, Radio Serra Azul De Frequenci...",230,322376890,<NA>,256.0
821189,94.1,-10.0733333333333333,-67.0647222222221666,"[MOS] FM-C4, C, Radio, Tv E Jornal Impresso Am...",230,693210850,<NA>,256.0
821190,96.7,-4.448410000000000,-49.112800000000000,"[MOS] FM-C4, B1, Rede Norte De Comunicacao Ltd...",230,1007534009,<NA>,256.0


In [ ]:
df_right = pd.read_parquet(folder / 'aero.parquet.gzip')

In [ ]:
df_right.tail()

,Frequency,Latitude,Longitude,Description
3748,1090.0,2.22156,-55.94504928588867,[DOC] Radar secundário (Airbone SSR)
3749,1090.0,-5.53668,-47.460601806640625,[DOC] Radar secundário (Airbone SSR)
3750,1090.0,-6.64257,-51.95793151855469,[DOC] Radar secundário (Airbone SSR)
3751,1090.0,-11.61905,-50.69247055053711,[DOC] Radar secundário (Airbone SSR)
3752,1090.0,-9.36829,-40.56911849975586,[DOC] Radar secundário (Airbone SSR)


In [ ]:
on: str = "Frequency"  # Coluna usada como chave de mesclagem
coords: Tuple[str] = ("Latitude", "Longitude")
description: str = "Description"
suffixes: Tuple[str] = ("_x", "_y")  # Sufixo para as colunas que foram criadas

df: pd.DataFrame = pd.merge(
        df_left.astype("string"),
        df_right.astype("string"),
        on=on,
        how="outer",
        suffixes=suffixes,
        indicator=True,
        copy=False,
    )

x, y = suffixes
lat, long = coords


left = df._merge == "left_only"
right = df._merge == "right_only"
both = df._merge == "both"
df = df.drop(columns=["_merge"])
left_cols: List[str] = [c for c in df.columns if y not in c]
right_cols: List[str] = [c for c in df.columns if x not in c]


only_left = df.loc[left, left_cols].drop_duplicates().reset_index(drop=True)
only_left.columns = [c.removesuffix(x) for c in left_cols]

only_right = df.loc[right, right_cols].drop_duplicates().reset_index(drop=True)
only_right.columns = [c.removesuffix(y) for c in right_cols]

both_columns = [f"{lat}{x}", f"{long}{x}", f"{lat}{y}", f"{long}{y}"]

df.loc[both, "Distance"] = df.loc[both, both_columns].apply(get_km_distance, axis=1)

close = df.loc[both, "Distance"] <= MAX_DIST
df_close = df.loc[(both & close)].drop_duplicates().reset_index(drop=True)
df_close[f"{description}{x}"] = (
    df_close[f"{description}{x}"] + " | " + df_close[f"{description}{y}"]
)
df_close = df_close[left_cols]
df_close.columns = only_left.columns


In [ ]:
only_left

,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
0,0.028,-22.662778,-43.476389,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557670,J9E,8.0
1,0.03,-18.41,-49.1,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,859966,J3E,1.0
2,0.03,-22.774167,-47.004444,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,859753,J3E,1.0
3,0.03,-22.926667,-43.265,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,859761,J3E,0.5
4,0.03,-23.441667,-46.590833,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557823,J3E,1.0
...,...,...,...,...,...,...,...,...
818646,0.405,-22.876700000000000,-43.227900000000000,"[MOS] L, RC, Seadrill Servicos De Petroleo Ltd...",108,1003009783,A2A,2.04
818647,0.405,-22.876700000000000,-43.227900000000000,"[MOS] L, RC, Servicos De Petroleo Constellatio...",108,1006343340,A2A,2.04
818648,0.426,-22.876700000000000,-43.227900000000000,"[MOS] L, RC, Osm Do Brasil Gerenciamento De Op...",108,1007360035,A2A,2.04
818649,1.718,-22.876700000000000,-43.227900000000000,"[MOS] L, RC, Subsea7 Do Brasil Servicos Ltda (...",108,1008633507,A2A,2.04


In [ ]:
only_right

,Frequency,Service,Station,Class,BW,Latitude,Longitude,Description
0,113.4,<NA>,<NA>,<NA>,<NA>,-9.866667,-56.099998474121094,"[ICAO] VOR/DME, ALTA FLORESTA"
1,113.4,<NA>,<NA>,<NA>,<NA>,-9.868361,-56.104949951171875,[AISG] VOR - ALTA FLORESTA CH 81X OPR INFRAERO
2,113.4,<NA>,<NA>,<NA>,<NA>,-3.8567445,-32.42993927001953,[AISG] VOR - NORONHA CH 81X
3,113.4,<NA>,<NA>,<NA>,<NA>,-27.669916,-48.54045104980469,[AISG] VOR - FLORIANOPOLIS CH 81X
4,113.4,<NA>,<NA>,<NA>,<NA>,-9.866667,-56.099998474121094,"[ICAO] VOR/DME, ALTA FLORESTA | [AISG] VOR - A..."
...,...,...,...,...,...,...,...,...
1014,1090.0,<NA>,<NA>,<NA>,<NA>,2.22156,-55.94504928588867,[DOC] Radar secundário (Airbone SSR)
1015,1090.0,<NA>,<NA>,<NA>,<NA>,-5.53668,-47.460601806640625,[DOC] Radar secundário (Airbone SSR)
1016,1090.0,<NA>,<NA>,<NA>,<NA>,-6.64257,-51.95793151855469,[DOC] Radar secundário (Airbone SSR)
1017,1090.0,<NA>,<NA>,<NA>,<NA>,-11.61905,-50.69247055053711,[DOC] Radar secundário (Airbone SSR)


In [ ]:
df[both]

,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y,Distance
679,0.2,-19.415847,-42.548231,"[MOS] L, OP, Mge Transmissao S.A. (50410917141...",019,1001992978,J3D,4.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,391.489026
680,0.2,-22.916111,-43.768611,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557637,J9E,8.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,174.095832
681,0.2,-24.101667,-54.226667,"[MOS] L, OP, Companhia De Geracao Termica De E...",019,2525658,J3W,4.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,1247.860843
682,0.2,-5.198056,-49.016944,"[MOS] L, OP, Centrais Eletricas Do Norte Do Br...",019,354635,A3E,8.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,2098.546077
750,0.225,-20.193889,-40.290833,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,860034,J3E,1.0,-25.408667,-54.621665954589844,[AISG] NDB - ITAIPU,1579.384076
...,...,...,...,...,...,...,...,...,...,...,...,...
882769,0.31,-23.995100000000000,-46.249000000000000,"[MOS] L, RC, Sapura Navegacao Maritima S.A. (5...",108,1002356390,A2A,2.1,-16.703382,-43.81910705566406,[AISG] NDB - MONTES CLAROS COVERAGE 50NM OPR N...,846.123184
882770,0.31,-23.995100000000000,-46.249000000000000,"[MOS] L, RC, Sapura Navegacao Maritima S.A. (5...",108,1002356390,A2A,2.04,-16.703382,-43.81910705566406,[AISG] NDB - MONTES CLAROS COVERAGE 50NM OPR N...,846.123184
882771,0.31,-22.876700000000000,-43.227900000000000,"[MOS] L, RC, Sapura Navegacao Maritima S.A. (5...",108,1002956207,A2A,2.04,-16.703382,-43.81910705566406,[AISG] NDB - MONTES CLAROS COVERAGE 50NM OPR N...,686.200154
882772,0.31,-22.876700000000000,-43.227900000000000,"[MOS] L, RC, Sapura Navegacao Maritima S.A. (5...",108,1002956223,A2A,2.04,-16.703382,-43.81910705566406,[AISG] NDB - MONTES CLAROS COVERAGE 50NM OPR N...,686.200154


In [ ]:
df_close

,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
0,0.28,-28.280791666666666,-54.170222222222165,"[MOS] L, RC, Paula Gabriela Dalla Porta (50413...",108,1002970897,A2A,2.0
1,0.28,-28.678911111111,-49.06675833333333,"[MOS] L, RC, Secretaria De Estado Da Infra-Est...",108,1000481252,A2A,2.0
2,118.0,-10.294167,-48.359444,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699526159,A3E,6.0
3,118.0,-10.294167,-48.359444,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699526159,A3E,6.0
4,118.0,-10.294167,-48.359444,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699526159,A3E,6.0
...,...,...,...,...,...,...,...,...
2177,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0
2178,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0
2179,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0
2180,0.205,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,A2A,2.7


In [ ]:
df_close.drop_duplicates()

,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
0,0.28,-28.280791666666666,-54.170222222222165,"[MOS] L, RC, Paula Gabriela Dalla Porta (50413...",108,1002970897,A2A,2.0
1,0.28,-28.678911111111,-49.06675833333333,"[MOS] L, RC, Secretaria De Estado Da Infra-Est...",108,1000481252,A2A,2.0
2,118.0,-10.294167,-48.359444,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699526159,A3E,6.0
3,118.0,-10.294167,-48.359444,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699526159,A3E,6.0
4,118.0,-10.294167,-48.359444,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699526159,A3E,6.0
...,...,...,...,...,...,...,...,...
2177,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0
2178,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0
2179,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0
2180,0.205,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,A2A,2.7


In [ ]:
df_close.iloc[4].Description

'[MOS] L, FA, Empresa Brasileira De Infra-Estrutura Aeroportuaria (11030016470, 699526159), Palmas/TO | [ICAO] TWR 25/40, PALMAS | [AISW] SBPJ-COM, Torre, Brigadeiro Lysias Rodrigues'

In [ ]:
df_right.to_excel(folder / 'aero.xlsx', index=False)

In [ ]:
subset=['Frequency', 'Latitude', 'Longitude']

In [ ]:
from extracao.icao import *
icao = get_icao()
icao

,Frequency,Latitude,Longitude,Description
0,113.4,-9.866666666666667,-56.1,"[ICAO] VOR/DME, ALTA FLORESTA"
1,113.2,-3.25,-52.25,"[ICAO] VOR/DME, ALTAMIRA"
2,117.5,-4.183333333333334,-69.93333333333334,"[ICAO] VOR/DME, AMAZONICA"
3,115.4,-16.25,-49.0,"[ICAO] VOR/DME, ANAPOLIS"
4,112.0,-10.983333333333333,-37.06666666666667,"[ICAO] VOR/DME, ARACAJU STA. MARIA"
...,...,...,...,...
2626,1176,-20.766666666666666,-51.55,"[DOC] VOR/DME, URUBUPUNGA CASTILHO (Ground-bas..."
2627,1082.0,-12.7,-60.083333333333336,"[DOC] VOR/DME, VILHENA (Airbone DME)"
2628,1019,-12.7,-60.083333333333336,"[DOC] VOR/DME, VILHENA (Ground-based DME)"
2629,1126.0,-20.25,-40.28333333333333,"[DOC] VOR/DME, VITORIA GOIABEIRAS (Airbone DME)"


In [ ]:
icao.drop_duplicates(subset, ignore_index=True)

,Frequency,Latitude,Longitude,Description
0,113.4,-9.866666666666667,-56.1,"[ICAO] VOR/DME, ALTA FLORESTA"
1,113.2,-3.25,-52.25,"[ICAO] VOR/DME, ALTAMIRA"
2,117.5,-4.183333333333334,-69.93333333333334,"[ICAO] VOR/DME, AMAZONICA"
3,115.4,-16.25,-49.0,"[ICAO] VOR/DME, ANAPOLIS"
4,112.0,-10.983333333333333,-37.06666666666667,"[ICAO] VOR/DME, ARACAJU STA. MARIA"
...,...,...,...,...
2399,1176,-20.766666666666666,-51.55,"[DOC] VOR/DME, URUBUPUNGA CASTILHO (Ground-bas..."
2400,1082.0,-12.7,-60.083333333333336,"[DOC] VOR/DME, VILHENA (Airbone DME)"
2401,1019,-12.7,-60.083333333333336,"[DOC] VOR/DME, VILHENA (Ground-based DME)"
2402,1126.0,-20.25,-40.28333333333333,"[DOC] VOR/DME, VITORIA GOIABEIRAS (Airbone DME)"


In [ ]:
from extracao.aisgeo import *
aisg = get_aisg()
aisg

c:\Users\rsilva\OneDrive - ANATEL\Code\anateldb\extracao\aisgeo.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Channel"] = df[cols[0]].astype("int").astype("string") + df[cols[1]]
c:\Users\rsilva\OneDrive - ANATEL\Code\anateldb\extracao\aisgeo.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Frequency"] = -1.0


,Frequency,Latitude,Longitude,Description
0,0.265,-21.139333333,-50.426666667,[AISG] NDB - ARAÇATUBA OPR TAM
1,0.3,-25.402666667,-49.229,"[AISG] NDB - BACACHERI FM ARP, COVERAGE 50NM"
2,0.38,-22.314,-49.107166667,[AISG] NDB - BAURU COVERAGE 50NM OPR INFRAERO
3,0.23,-7.266,-35.892666667,[AISG] NDB - CAMPINA GRANDE COVERAGE 60NM OPR ...
4,0.42,-20.813166667,-49.406166667,[AISG] NDB - RIO PRETO OPR DAESP
...,...,...,...,...
148,1175.0,-25.537760556,-48.529854722,[AISG] DME - PARANAGUÁ 88X
149,1181.0,-18.203235,-45.45707222,[AISG] DME - TRÊS MARIAS 94X
150,1171.0,-27.621855278,-48.632463889,[AISG] DME - BIGUAÇU 84X
151,1186.0,-14.907780556,-40.918838889,[AISG] DME - VITÓRIA DA CONQUISTA 99X


In [ ]:
aisg.drop_duplicates(subset, ignore_index=True)

,Frequency,Latitude,Longitude,Description
0,0.265,-21.139333333,-50.426666667,[AISG] NDB - ARAÇATUBA OPR TAM
1,0.3,-25.402666667,-49.229,"[AISG] NDB - BACACHERI FM ARP, COVERAGE 50NM"
2,0.38,-22.314,-49.107166667,[AISG] NDB - BAURU COVERAGE 50NM OPR INFRAERO
3,0.23,-7.266,-35.892666667,[AISG] NDB - CAMPINA GRANDE COVERAGE 60NM OPR ...
4,0.42,-20.813166667,-49.406166667,[AISG] NDB - RIO PRETO OPR DAESP
...,...,...,...,...
268,1175.0,-25.537760556,-48.529854722,[AISG] DME - PARANAGUÁ 88X
269,1181.0,-18.203235,-45.45707222,[AISG] DME - TRÊS MARIAS 94X
270,1171.0,-27.621855278,-48.632463889,[AISG] DME - BIGUAÇU 84X
271,1186.0,-14.907780556,-40.918838889,[AISG] DME - VITÓRIA DA CONQUISTA 99X


In [ ]:
from extracao.aisweb import *
aisw = get_aisw()
aisw

,Frequency,Latitude,Longitude,Description
0,118.6,-3.145833333333,-59.985,"[AISW] SBMN-COM, Torre, Campo Ponta Pelada"
1,109.3,-3.1502777777777777,-59.98222222222223,"[AISW] SBMN-NAV, ILS/DME 09 IPE, Campo Ponta P..."
0,118.5,-29.945555555556,-51.143611111111,"[AISW] SBCO-COM, Torre, CAMPO NOSSA SENHORA DE..."
1,122.8,-29.945555555556,-51.143611111111,"[AISW] SBCO-COM, Torre, CAMPO NOSSA SENHORA DE..."
2,121.7,-29.945555555556,-51.143611111111,"[AISW] SBCO-COM, Solo, CAMPO NOSSA SENHORA DE ..."
...,...,...,...,...
502,1054.0,-22.796666666666667,-43.223888888888894,"[DOC] SBGL-NAV, ILS 10 CAT II, Galeão - Antôni..."
503,991,-22.796666666666667,-43.223888888888894,"[DOC] SBGL-NAV, ILS 10 CAT II, Galeão - Antôni..."
504,332.9,-22.804444444444446,-43.26416666666667,"[DOC] SBGL-NAV, ILS/DME 28 ILM, Galeão - Antôn..."
505,1076.0,-22.804444444444446,-43.26416666666667,"[DOC] SBGL-NAV, ILS/DME 28 ILM, Galeão - Antôn..."


In [ ]:
aisw.drop_duplicates(subset, ignore_index=True)

,Frequency,Latitude,Longitude,Description
0,118.6,-3.145833333333,-59.985,"[AISW] SBMN-COM, Torre, Campo Ponta Pelada"
1,109.3,-3.1502777777777777,-59.98222222222223,"[AISW] SBMN-NAV, ILS/DME 09 IPE, Campo Ponta P..."
2,118.5,-29.945555555556,-51.143611111111,"[AISW] SBCO-COM, Torre, CAMPO NOSSA SENHORA DE..."
3,122.8,-29.945555555556,-51.143611111111,"[AISW] SBCO-COM, Torre, CAMPO NOSSA SENHORA DE..."
4,121.7,-29.945555555556,-51.143611111111,"[AISW] SBCO-COM, Solo, CAMPO NOSSA SENHORA DE ..."
...,...,...,...,...
489,1054.0,-22.796666666666667,-43.223888888888894,"[DOC] SBGL-NAV, ILS 10 CAT II, Galeão - Antôni..."
490,991,-22.796666666666667,-43.223888888888894,"[DOC] SBGL-NAV, ILS 10 CAT II, Galeão - Antôni..."
491,332.9,-22.804444444444446,-43.26416666666667,"[DOC] SBGL-NAV, ILS/DME 28 ILM, Galeão - Antôn..."
492,1076.0,-22.804444444444446,-43.26416666666667,"[DOC] SBGL-NAV, ILS/DME 28 ILM, Galeão - Antôn..."


In [ ]:
from extracao.redemet import *
rmet = get_redemet()
rmet

,Frequency,Latitude,Longitude,Description
0,2800,-16.201531,-40.674153,[RMET] Radar - Almenara/MG
1,2800,-1.406667,-48.461389,[RMET] Radar - Belém/PA
2,2800,2.844166667,-60.70027777778,[RMET] Radar - Boa Vista/RR
3,2800,-31.404,-52.701644,[RMET] Radar - Canguçu/RS
4,2800,-7.595833,-72.767778,[RMET] Radar - Cruzeiro do Sul/AC
5,2800,-15.97643,-48.016142,[RMET] Radar - Gama/DF
6,2800,-20.27855,-54.47396,[RMET] Radar - Jaraguari/MS
7,2800,-3.149216,-59.991881,[RMET] Radar - Manaus/AM
8,2800,-0.047222,-51.097778,[RMET] Radar - Macapá/AP
9,2800,-9.55129,-35.77068,[RMET] Radar - Maceió/AL


In [ ]:
rmet.drop_duplicates(subset, ignore_index=True)

,Frequency,Latitude,Longitude,Description
0,2800,-16.201531,-40.674153,[RMET] Radar - Almenara/MG
1,2800,-1.406667,-48.461389,[RMET] Radar - Belém/PA
2,2800,2.844166667,-60.70027777778,[RMET] Radar - Boa Vista/RR
3,2800,-31.404,-52.701644,[RMET] Radar - Canguçu/RS
4,2800,-7.595833,-72.767778,[RMET] Radar - Cruzeiro do Sul/AC
5,2800,-15.97643,-48.016142,[RMET] Radar - Gama/DF
6,2800,-20.27855,-54.47396,[RMET] Radar - Jaraguari/MS
7,2800,-3.149216,-59.991881,[RMET] Radar - Manaus/AM
8,2800,-0.047222,-51.097778,[RMET] Radar - Macapá/AP
9,2800,-9.55129,-35.77068,[RMET] Radar - Maceió/AL
